### Import modules

In [1]:
import os
import sys
import time
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import arcpy
import functions_warcpy as fwa

### Initialize variables

In [2]:
proj_dir = input("Path to project directory (e.g. \\\Mac\stor\Projects\TransectExtraction\FireIsland2014): ")

Path to project directory (e.g. \\Mac\stor\Projects\TransectExtraction\FireIsland2014): \\Mac\stor\Projects\TransectExtraction\FireIsland2014


In [3]:
site = input("site: ")
year = input("year: ")

site: FireIsland
year: 2014


In [4]:
from setvars import *

## Dunes and armoring
Display the points and the DEM in a GIS to check for irregularities. For example, if shoreline points representing a distance less than X m are visually offset from the general shoreline, they should likely be removed. Another red flag is when the positions of dlows and dhighs in relation to the shore are illogical, i.e. dune crests are seaward of dune toes. 

#### Replace fill values with Null. 

In [ ]:
fwa.ReplaceValueInFC(dhPts, oldvalue=fill, newvalue=None, fields=["dhigh_z"])      # Dhighs
fwa.ReplaceValueInFC(dlPts, oldvalue=fill, newvalue=None, fields=["dlow_z"])       # Dlows
fwa.ReplaceValueInFC(ShorelinePts, oldvalue=fill, newvalue=None, fields=["slope"]) # Shoreline

#### Armoring
If the dlows do not capture the entire top-of-beach due to atypical formations caused by anthropogenic modification, you may need to digitize the beachfront armoring. The next code block will generate an empty feature class. Refer to the DEM and orthoimagery. If there is no armoring in the study area, continue. If there is armoring, use the Editing toolbar to add lines to the feature class that trace instances of armoring. Common manifestations of what we call armoring are sandfencing and sandbagging and concrete seawalls. 

*__Requires manipulation in GIS__*

In [ ]:
arcpy.CreateFeatureclass_management(home, armorLines, 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now manually digitize the shorefront armoring.".format(armorLines))

## Inlets
We also need to manually digitize inlets if an inlet delineation does not already exist. To do, the code will produce the feature class. After which use the Editing toolbar to create a line when the island shore meets a tidal inlet. If the study area includes both sides of an inlet, that inlet will be represented by two lines. The inlet lines are use to define the bounds of the oceanside shore, which is also considered the point where the oceanside shore meets the bayside. Inlet lines must intersect the MHW contour. 

*__Requires manipulation in GIS__*

In [ ]:
# manually create lines based on the boundary polygon that correspond to end of land and cross the MHW line
arcpy.CreateFeatureclass_management(home, inletLines, 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now we'll stop for you to manually create lines at each inlet.".format(inletLines))

## Shoreline
The shoreline is produced through a combination of the DEM and the shoreline points. The first step converts the DEM to both MTL and MHW contour polygons. Those polygons are combined to produce the full shoreline, which is considered to fall at MHW on the oceanside and MTL on the bayside (to include partially submerged wetland). User input is required to identify only the areas within the study area and eliminate isolated landmasses that are not. 

*__Requires display in GIS__*

In [ ]:
#%% BOUNDARY POLYGON
# Inlet lines must intersect MHW
bndpoly = fwa.DEMtoFullShorelinePoly(elevGrid_5m, '{site}{year}'.format(**SiteYear_strings), MTL, MHW, inletLines, ShorelinePts)
# Eliminate any remnant polygons on oceanside
print('Select features from {} that should not be included in {}'.format(bndpoly, barrierBoundary))

Once the features to delete are selected, either delete in the GIS or run the code.

In [ ]:
arcpy.DeleteFeatures_management(bndpoly)

The next step snaps the boundary polygon to the shoreline points anywhere they don't already match and as long as as they are within 25 m of each other. 

In [ ]:
barrierBoundary = fwa.NewBNDpoly(bndpoly, ShorelinePts, barrierBoundary, '25 METERS', '50 METERS')
shoreline = fwa.CreateShoreBetweenInlets(barrierBoundary, inletLines, shoreline, ShorelinePts, proj_code)

## Transects
### Extended transects

Create extendedTrans, LT transects with gaps filled and lines extended

Set temp filenames.

In [8]:
trans_presort = os.path.join(arcpy.env.scratchGDB, 'trans_presort_temp')
trans_extended = os.path.join(arcpy.env.scratchGDB, 'trans_ext_temp')
trans_sort_1 = os.path.join(arcpy.env.scratchGDB, 'trans_sort_temp')
trans_x = os.path.join(arcpy.env.scratchGDB, 'overlap_points_temp')
overlapTrans_lines = os.path.join(arcpy.env.scratchGDB, 'overlapTrans_lines_temp')
sort_lines =  os.path.join(arcpy.env.scratchGDB, 'sort_lines')

OSError: "FireIsland_LTorig" does not exist

#### 1. Extend and Copy only the geometry of transects to use as material for filling gaps

In [ ]:
fwa.ExtendLine(fc=orig_trans, new_fc=trans_extended, distance=extendlength, proj_code=proj_code)
fwa.CopyAndWipeFC(trans_extended, trans_presort, ['sort_ID'])
print("MANUALLY: use groups of existing transects in new FC '{}' to fill gaps.".format(trans_presort))

*__Requires manipulation in GIS__*

Manually use groups of existing transects in the new FC to fill gaps. Avoid overlapping transects as much as possible. The new FC is only for use as spatial filler. As a result, the attribute values don't matter and there is no need to preserve the original transects (i.e. you can simply move transects instead of duplicating them). However, do not slightly move any transects. If they are moved, they will not be deleted in the next stage. If you slightly move any, you can eithe undo or delete that line entirely.

#### 2. Automatically sort the transects.

First set the sorting parameters. If the shoreline curves, the GIS will not correctly establish the alongshore order by simple ordering from the identified sort_corner so you need to identify different groups of transects for sorting. If this is the case, answer yes to the next prompt.

In [25]:
multi_sort = input("Do we need to sort the transects in batches to preserve the order? (yes/no) ") 

Do we need to sort the transects in batches to preserve the order? (yes/no) yes
Sort corner (LL, LR, UL, UR): LL


In [ ]:
fwa.PrepTransects_part2(trans_presort, trans_extended, barrierBoundary)
# Create lines to use to sort new transects
if multi_sort == 'yes':
    arcpy.CreateFeatureclass_management(trans_dir, sort_lines, "POLYLINE", spatial_reference=utmSR)
    print("MANUALLY: Add features to sort_lines. Add the features in the order they should dictate sorting and indicate the sort corner in the field sort_corn.")
else:
    sort_lines = []
    sort_corner = input("Sort corner (LL, LR, UL, UR): ")   # Corner from which to start sorting, LL = lower left, etc. 

*__Requires manipulation in GIS__*

If the transects need to be sorted in batches to preserve the order, then we generated an empty sort lines feature class. Add lines to the FC that will be used to iteratively select the intersecting transects and order them based on the sort corner you will indicate next. Add fields sort and sort_corn to the FC and assign values to order the transects. sort_corn indicates the sort corner for the group pertaining to each line. 

In [ ]:
fwa.SortTransectsFromSortLines(trans_presort, extendedTrans, sort_lines, sortfield=tID_fld, sort_corner=sort_corner)
# # Clean up OBJECTID
# if len(arcpy.ListFields(extendedTrans, 'OBJECTID*')) == 2:
#     fwa.ReplaceFields(extendedTrans, {'OBJECTID': 'OID@'})

### Tidy extended transects

If extended transects overlap, they need to be semi-manually adjusted ('tidied') so that there is no overlap. Overlapping transects cause problems when splitting the transects into 5-m points and when converting the transects to rasters. The user needs to select the boundary lines between groups of overlapping transects. 

*__Requires manipulation in GIS__*

After the boundary lines have been selected, those lines are copied into a separate FC, which is then used to split the transects at the overlapping points. Once the lines are split, the user needs to manually delete the undesired pieces. These will be the ones that are to the bayside of the boundary lines.

In [ ]:
# Copy only the selected lines
arcpy.CopyFeatures_management(orig_extTrans, overlapTrans_lines)
arcpy.SelectLayerByAttribute_management(orig_extTrans, "CLEAR_SELECTION")

# Split transects at the lines of overlap.
arcpy.Intersect_analysis([orig_extTrans, overlapTrans_lines], trans_x,
                         'ALL', output_type="POINT")
arcpy.SplitLineAtPoint_management(orig_extTrans, trans_x, extTrans_tidy)
print("MANUALLY: Select transect segments to be deleted. ")

Resume after manual selection

In [ ]:
arcpy.DeleteFeatures_management(extTrans_tidy)
print("Pre-processing completed.")